In [2]:
import pandas as pd

jigsaw_all = pd.read_csv('data/all_data.csv')
jigsaw_toxic_idxs = pd.read_csv('data/toxicity_individual_annotations.csv')
obscenity_list = pd.read_csv('data/profanity_en.csv')
reddit_one_mil = pd.read_csv('data/kaggle_RC_2019-05.csv')

In [108]:
sexually_explicit_comments = pd.DataFrame({'body': [], 'source': []})

In [109]:
## data from r/gonewild

with open('data/sexually_explicit.csv', 'r', encoding='utf-8') as f:
    lines = f.readlines()

temp_df = pd.DataFrame({'body': lines, 'source': ['gonewild'] * len(lines)})
sexually_explicit_comments = pd.concat([sexually_explicit_comments, temp_df])

In [3]:
## data from jigsaw

data = jigsaw_all[jigsaw_all['sexual_explicit'] >= 0.5].comment_text.tolist()
temp_df = pd.DataFrame({'body': data, 'source': ['jigsaw'] * len(data)})
# sexually_explicit_comments = pd.concat([sexually_explicit_comments, temp_df])
# sexually_explicit_comments
temp_df

,body,source
0,"""while arresting a man for resisting arrest"".\...",jigsaw
1,What an ugly orgy. Shame on Fish and Game for...,jigsaw
2,Headline Porn!,jigsaw
3,no more government money into the media. They ...,jigsaw
4,"Deputy Cox is a sadist, his colleagues who wit...",jigsaw
...,...,...
5122,A lot of stereotyping in your post. Women are ...,jigsaw
5123,People can't just get along with their lives. ...,jigsaw
5124,A visit to Thailand will suffice. I have been ...,jigsaw
5125,I'm not one for engaging in semantics argument...,jigsaw


In [84]:
def filter_comments(row):
    comment = row['body'].lower()
    long_enough = len(comment.split()) >= 3
    not_a_bot = not(comment.startswith("your submission") or comment.startswith("[**") or comment.startswith("**") or comment.startswith("^(") or comment.startswith('> ') or ("i'm a bot" in comment) or ("i am a bot" in comment))
    return long_enough and not_a_bot

In [111]:
## data from one million reddit comments r/gonewild

data = reddit_one_mil[reddit_one_mil['subreddit'] == 'gonewild'].body.tolist()
temp_df = pd.DataFrame({'body': data, 'source': ['gonewild'] * len(data)})
sexually_explicit_comments = pd.concat([sexually_explicit_comments, temp_df])

In [112]:
## removing comments less than 3 words or produced by bots

filtered_target_df = sexually_explicit_comments[sexually_explicit_comments.apply(filter_comments, axis=1)]
filtered_target_df

print(len(sexually_explicit_comments) - len(filtered_target_df))

2830


In [113]:
## general text data, ignoring non-english subreddits

non_target_df = reddit_one_mil[~reddit_one_mil['subreddit'].isin(['gonewild', 'Pikabu'])]
non_target_df = non_target_df.drop(['controversiality', 'score'], axis=1)
non_target_df = non_target_df.rename(columns={'subreddit': 'source'})

non_target_df

,source,body
0,gameofthrones,Your submission has been automatically removed...
1,aww,"Dont squeeze her with you massive hand, you me..."
2,gaming,It's pretty well known and it was a paid produ...
3,news,You know we have laws against that currently c...
4,politics,"Yes, there is a difference between gentle supp..."
...,...,...
999868,trashy,They are fighting a violent regime
999870,trashy,LOL. He also said something about turning them...
999871,trashy,Yeah I love helping and being close to my fami...
999876,trashy,As a Canadian lefty I have to say I find Ameri...


In [114]:
## same filtering process

non_target_df = non_target_df[non_target_df.apply(filter_comments, axis=1)]
non_target_df

,source,body
1,aww,"Dont squeeze her with you massive hand, you me..."
2,gaming,It's pretty well known and it was a paid produ...
3,news,You know we have laws against that currently c...
4,politics,"Yes, there is a difference between gentle supp..."
5,dankmemes,Please let this become a meme. I want some Dar...
...,...,...
999868,trashy,They are fighting a violent regime
999870,trashy,LOL. He also said something about turning them...
999871,trashy,Yeah I love helping and being close to my fami...
999876,trashy,As a Canadian lefty I have to say I find Ameri...


In [115]:
# final data
filtered_target_df['label'] = 1
non_target_df['label'] = 0


C:\Users\vince\AppData\Local\Temp\ipykernel_31292\472895026.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_target_df['label'] = 1
C:\Users\vince\AppData\Local\Temp\ipykernel_31292\472895026.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_target_df['label'] = 0


In [ ]:
# Combine the two DataFrames
df = pd.concat([filtered_target_df, non_target_df], ignore_index=True)

# Write to CSV
df.to_csv('sexually_explicit_text.csv', index=False)

In [116]:
import numpy as np
np.random.seed(0)

## downsampled version
downsample_target = len(filtered_target_df)
choose_idxs = np.random.permutation(len(non_target_df))
choose_idxs = choose_idxs[:downsample_target]

In [120]:
downsampled_non_targets = non_target_df.iloc[choose_idxs]
downsampled_df = pd.concat([filtered_target_df, downsampled_non_targets])
downsampled_df.to_csv('sexually_explicit_text_downsampled.csv', index=False)
